In [5]:
import random

text_file = "/home/binbin/dl/python_deep_learning/spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t") 
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))


print(random.choice(text_pairs))

('Make it simple.', '[start] Simplifica. [end]')


In [8]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]



In [13]:
import tensorflow as tf
import string
import re
import keras 
from keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

I0000 00:00:1734783416.901369   47792 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734783417.281651   47792 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734783417.281701   47792 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734783417.286683   47792 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734783417.286786   47792 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [14]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1]
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [16]:
for inputs, targets in train_ds.take(1):
     print(f"inputs['english'].shape: {inputs['english'].shape}")
     print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
     print(f"targets.shape: {targets.shape}")


inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


2024-12-21 20:18:02.638945: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-12-21 20:18:02.639853: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [21]:
# inputs = keras.Input(shape=(sequence_length,), dtype="int64")
# x = layers.Embedding(input_dim=vocab_size, output_dim=128)(inputs)
# x = layers.LSTM(32, return_sequences=True)(x)
# outputs = layers.Dense(vocab_size, activation="softmax")(x)
# model = keras.Model(inputs, outputs)

embed_dim = 256
latent_dim = 512

source = keras.Input(shape=(None,), dtype="int64", name="english") 
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source) 
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)


past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source) 
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [ ]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15


2024-12-21 20:26:18.974112: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1302/1302 ━━━━━━━━━━━━━━━━━━━━ 144s 109ms/step - accuracy: 0.1351 - loss: 5.3538 - val_accuracy: 0.1462 - val_loss: 4.1379
Epoch 2/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 144s 111ms/step - accuracy: 0.1508 - loss: 4.0972 - val_accuracy: 0.1759 - val_loss: 3.5275
Epoch 3/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 143s 110ms/step - accuracy: 0.1732 - loss: 3.6139 - val_accuracy: 0.1935 - val_loss: 3.1935
Epoch 4/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 142s 109ms/step - accuracy: 0.1879 - loss: 3.2955 - val_accuracy: 0.2067 - val_loss: 2.9473
Epoch 5/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 143s 110ms/step - accuracy: 0.1990 - loss: 3.0500 - val_accuracy: 0.2169 - val_loss: 2.7681
Epoch 6/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 145s 111ms/step - accuracy: 0.2091 - loss: 2.8480 - val_accuracy: 0.2252 - val_loss: 2.6183
Epoch 7/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 144s 111ms/step - accuracy: 0.2176 - loss: 2.6797 - val_accuracy: 0.2317 - val_loss: 2.5128
Epoch 8/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 144s 111ms/step - accuracy: 0.2

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))